<a href="https://colab.research.google.com/github/lannd3217/Advanced-Business-Analysis/blob/main/InterviewRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/lannd3217/Interview_RAG.git

In [ ]:
from google.colab import userdata
import os

GIT_TOKEN = userdata.get('GITHUB_TOKEN')
GIT_USER = "lannd3217"
GIT_REPO = "Interview_RAG"
GIT_EMAIL = "lanngocd.17@gmail.com"
GIT_NAME = "LAN DINH"

!git config --global user.email {GIT_EMAIL}
!git config --global user.name {GIT_NAME}
remote_url = f"https://{GIT_TOKEN}@github.com/{GIT_USER}/{GIT_REPO}.git"
!git clone {remote_url}

In [ ]:
%ls

In [ ]:
%cd "Interview_RAG"

In [ ]:
%%capture
!pip install -U langchain langchain-community langchain-openai
!pip install -U langchain langchain-community langchain-text-splitters
!pip install -U pymupdf langchain-community
!pip install -U langchain-huggingface sentence-transformers
!pip install -q sentence-transformers faiss-cpu transformers
!pip install langchain langchain-community langchain-chroma langchain-huggingface pymupdf sentence-transformers
!pip install "unstructured[all-docs]"


In [ ]:
import os
import re
# Updated import path
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, CSVLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
## LOADING
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader, TextLoader

def load_docs_from_folder(folder_path="./docs"):
    """
    Loads all PDF and TXT documents from a specified directory.
    """
    print(f"Loading documents from: {folder_path}...")

    # Define the mapping of file extensions to loader classes
    loaders_mapping = {
        "*.pdf": PyMuPDFLoader,
        "*.txt": TextLoader
    }

    docs = []

    # We loop through the mapping to ensure each file type is handled correctly
    for glob_pattern, loader_class in loaders_mapping.items():
        loader = DirectoryLoader(
            folder_path,
            glob=glob_pattern,
            loader_cls=loader_class,
            show_progress=True,
            silent_errors=True
        )
        docs.extend(loader.load())

    print(f"Successfully loaded {len(docs)} document objects.")
    return docs

In [ ]:
docs = load_docs_from_folder("./docs")

In [ ]:
search_term = "A/B testing"
for doc in docs:
    if search_term.lower() in doc.page_content.lower():
        print(f"Found '{search_term}' in {doc.metadata['source']} on page {doc.metadata['page']}")

In [ ]:

## CLEANING

import re

def clean_text(text, is_reddit=False):
    """
    Unified cleaning function for both PDF and Reddit/HTML content.
    """
    # --- PART 1: Reddit/HTML Specific Cleaning ---
    if is_reddit:
        # Remove Top Navigation and Sidebar noise
        text = re.sub(r"Skip to main content.*?Go to datascience", "", text, flags=re.DOTALL)

        # Remove UI buttons and repetitive Reddit metadata
        ui_patterns = [
            r"Upvote", r"Downvote", r"Award", r"Share", r"Open chat",
            r"Create post", r"Open inbox", r"Expand user menu",
            r"More replies", r"1 more reply", r"Archived post.*?\.",
            r"Sort by:", r"Best", r"Search Comments", r"Comments Section",
            r"avatar", r"•", r"\d+mo ago", r"\d+h ago"
        ]
        for pattern in ui_patterns:
            text = re.sub(pattern, "", text, flags=re.IGNORECASE)

        # Normalize User Handles (e.g., u/User avatar User -> User: User)
        text = re.sub(r"u/(\w+)\s+avatar\s+\1", r"\nUser: \1\n", text)

    # --- PART 2: General/PDF Specific Cleaning ---
    # 1. Remove weird "cid" characters
    text = re.sub(r'\(cid:\d+\)', '', text)

    # 2. Remove purely numerical lines (page numbers/footers)
    text = re.sub(r'^\d+$\n', '', text, flags=re.MULTILINE)

    # --- PART 3: Whitespace Normalization (Apply to all) ---
    # Replace multiple newlines with a single one
    text = re.sub(r'\n\s*\n', '\n', text)
    # Replace multiple spaces with a single space
    text = re.sub(r' +', ' ', text)

    return text.strip()

# --- Apply cleaning and create the hashmap ---
cleaned_samples = {}

for doc in docs:
    # 1. Extract raw content
    raw_content = doc.page_content

    # 2. Get the filename from metadata
    filename = doc.metadata.get('source', 'Unknown File')

    # 3. Detect if it's a Reddit/Forum file
    is_reddit_file = "reddit" in filename.lower() or ".txt" in filename.lower()

    # 4. Clean the text
    cleaned_content = clean_text(raw_content, is_reddit=is_reddit_file)

    # 5. Store in hashmap (merging pages if the file has multiple)
    if filename in cleaned_samples:
        cleaned_samples[filename] += "\n" + cleaned_content
    else:
        cleaned_samples[filename] = cleaned_content

# --- Verify the results ---
# print(f"Success! Cleaned and indexed {len(cleaned_samples)} unique documents.")
# cleaned_samples["docs/Reddit_DS_NewGrad.txt"]
# --- Print the Example Output ---
print(f"Processed {len(cleaned_samples)} unique documents.\n")

# We convert the dictionary items to a list so we can slice the first 3
for filename, text in list(cleaned_samples.items())[:3]:
    print(f"Source: {filename}")
    print(f"--- Cleaned Text Preview ---\n{text[:350]}...")
    print("-" * 30)

In [ ]:
## SPLITTING

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # Roughly 200 words
    chunk_overlap=200, # Overlap helps keep context between chunks
    separators=["\n\n", "\n", " ", ""]
)

# Create the chunks
chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks from {len(cleaned_samples)} documents.")

In [ ]:
import os
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, CSVLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def clean_text(text, is_reddit=False):
    """Refined unified cleaning logic."""
    if is_reddit:
        # Remove Reddit-specific UI noise
        text = re.sub(r"Skip to main content.*?Go to datascience", "", text, flags=re.DOTALL)
        ui_patterns = [r"Upvote", r"Downvote", r"Award", r"Share", r"avatar", r"•", r"\d+mo ago"]
        for p in ui_patterns:
            text = re.sub(p, "", text, flags=re.IGNORECASE)

    text = re.sub(r'\(cid:\d+\)', '', text) # PDF artifacts
    text = re.sub(r'^\d+$\n', '', text, flags=re.MULTILINE) # Page numbers
    text = re.sub(r'\n\s*\n', '\n', text) # Normalize newlines
    text = re.sub(r' +', ' ', text) # Normalize spaces
    return text.strip()

def ingestion_pipeline(folder_path="./docs"):
    # 1. LOAD
    print(f"Starting ingestion from: {folder_path}")

    # Map extensions to specific loaders
    loaders_mapping = {
        "*.pdf": PyMuPDFLoader,
        "*.txt": TextLoader,
        "*.csv": CSVLoader
    }

    raw_docs = []
    for glob_pattern, loader_cls in loaders_mapping.items():
        # Only attempt to load if the folder actually contains these files
        loader = DirectoryLoader(
            folder_path,
            glob=glob_pattern,
            loader_cls=loader_cls,
            silent_errors=True
        )
        raw_docs.extend(loader.load())

    if not raw_docs:
        print("No documents found. Check your folder path or file extensions.")
        return []

    # 2. CLEAN & 4. ATTACH METADATA
    cleaned_docs_map = {}
    for doc in raw_docs:
        source = doc.metadata.get('source', 'Unknown')
        # Identify Reddit content for specialized cleaning
        is_reddit = "reddit" in source.lower() or source.endswith(".txt")

        cleaned_content = clean_text(doc.page_content, is_reddit=is_reddit)

        if source not in cleaned_docs_map:
            cleaned_docs_map[source] = {"text": "", "metadata": doc.metadata}
        cleaned_docs_map[source]["text"] += "\n" + cleaned_content

    # 3. SPLIT
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=100,
        length_function=len
    )

    final_document_objects = []
    doc_chunk_counts = {}

    for source, data in cleaned_docs_map.items():
        # Create temporary document for splitting
        temp_doc = Document(page_content=data["text"], metadata=data["metadata"])
        chunks = text_splitter.split_documents([temp_doc])

        # Enrich metadata
        for chunk in chunks:
            chunk.metadata["doc_name"] = os.path.basename(source)
            chunk.metadata["section"] = "Main Content"
            final_document_objects.append(chunk)

        doc_chunk_counts[os.path.basename(source)] = len(chunks)

    # 5. OUTPUT METRICS
    total_chunks = len(final_document_objects)
    avg_size = sum(len(d.page_content) for d in final_document_objects) / total_chunks if total_chunks > 0 else 0

    print("\n--- Ingestion Metrics ---")
    print(f"Total Chunks Created:  {total_chunks}")
    print(f"Average Chunk Size:    {avg_size:.2f} chars")
    print(f"Metadata Coverage:     100% (doc_name, section attached)")
    print("\n--- Chunks per Document ---")
    for name, count in doc_chunk_counts.items():
        print(f"- {name}: {count} chunks")

    return final_document_objects

# Execute
# Note: Ensure you have run !pip install langchain-core langchain-community pymupdf
docs_for_vectorstore = ingestion_pipeline("./docs")

In [ ]:
docs_for_vectorstore[3].page_content

In [ ]:
# ## EMBEDDING
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS

# # 1. Initialize the Embedding Model
# print("Downloading embedding model (MiniLM-L6)...")
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # 2. Create the Vector Store from your full document list
# print(f"Embedding {len(docs_for_vectorstore)} chunks into FAISS...")
# # This turns the text into high-dimensional vectors
# vector_db = FAISS.from_documents(docs_for_vectorstore, embeddings)

# # 3. Save the index locally
# vector_db.save_local("faiss_interview_index")
# print("FAISS Index saved and ready for retrieval!")

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma

# print("\nDownloading embedding model (MiniLM-L6)...")
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # 3. Create ChromaDB (Local-First Prototyping)
# print(f"Embedding and storing in ChromaDB...")
# vector_db = Chroma.from_documents(
#     documents=docs_for_vectorstore,
#     embedding=embeddings,
#     persist_directory="./interview_knowledge_base",
#     collection_name="interview_prep"
# )

# print(f"\n SUCCESS: Vector Store created with {vector_db._collection.count()} records.")

In [ ]:
# # Fetch a sample from your collection to inspect components
# sample_data = vector_db.get(limit=100, include=['embeddings', 'documents', 'metadatas'])

# # --- 1. CHECK EMBEDDINGS (The 384-dim vectors) ---
# embedding_sample = sample_data['embeddings'][50]
# print(f"Embeddings Found: Vector length is {len(embedding_sample)}")
# # Should be 384 if using all-MiniLM-L6-v2

# # --- 2. CHECK SOURCE TEXT (Original chunk) ---
# document_sample = sample_data['documents'][1]
# print(f"Source Text Found: '{document_sample[:50]}...'")

# # --- 3. CHECK METADATA (source_doc, page, section, etc.) ---
# metadata_sample = sample_data['metadatas'][1]
# print(f"Metadata Found: {metadata_sample.keys()}")
# # Look for: 'doc_name', 'section', 'source'

# # --- 4. CHECK IDs (Unique identifiers) ---
# id_sample = sample_data['ids'][0]
# print(f"Unique ID Found: {id_sample}")

In [ ]:
## EMBEDDING
## VECTOR DATABASE
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import uuid

client = chromadb.PersistentClient(path="./interview_vector_db")

embedding_func = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")


collection = client.get_or_create_collection(
    name="interview_prep_collection",
    embedding_function=embedding_func
)



In [ ]:
documents_text = [doc.page_content for doc in docs_for_vectorstore]
metadatas = [doc.metadata for doc in docs_for_vectorstore]
ids = [str(uuid.uuid4()) for _ in range(len(docs_for_vectorstore))]
# ids = [f"id{i}" for i in range(len(docs_for_vectorstore))]

print(f"Inserting {len(documents_text)} chunks into ChromaDB...")
collection.add(
    ids=ids,
    documents=documents_text,
    metadatas=metadatas
)

print(f"Collection created. Total items: {collection.count()}")

In [ ]:
# List all collections
print(client.list_collections())
# collection.peek()
collection.get(ids=['ab2c95cb-3f9b-4cec-92b1-8b7f06430faa'], include=["embeddings"])

In [ ]:
print("\n--- Component Audit ---")
peek_result = collection.peek(1)
print(f"ID: {peek_result['ids'][0]}")
print(f"Metadata: {peek_result['metadatas'][0]}")
print(f"Document Preview: {peek_result['documents'][0][:50]}...")

# B. Semantic Search Test (Slide: "Querying a Collection")
query = "How do I handle a technical interview for a data science role?"
results = collection.query(
    query_texts=[query],
    n_results=3
)

print(f"\nSemantic Search Results for: '{query}'")
for i in range(len(results['documents'][0])):
    print(f"Result {i+1} (Distance: {results['distances'][0][i]:.4f}):")
    print(f"Content: {results['documents'][0][i]}...")
    print("-" * 20)

In [ ]:
results

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma
# ## RETRIEVAL
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Instantiate Retriever
# vector_store = Chroma.from_documents(
#     documents = chunks,
#     embedding = embeddings
# )


In [ ]:
# retriever = vector_store.as_retriever(
#     search_type = "similarity",
#     search_kwargs = {"k":2}
# )

In [ ]:
# # Create Prompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPrompTemplate.from_template("""
# Use the following pievces of context to answer the question at the end.
# If you don't know the answer, say that you don't know.
# Context: {context}
# Question: {question}
# """)

In [ ]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model ="gpt-4o-mini",api_key=userdata.get('OPEN_AI'), temperature=0)

In [ ]:
# # Assemble the Chain
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser

# chain = (
#     {"context": retriever, "question": RunnablePassthrough}
#     | prompt
#     | llm
#     | StrOutputParser
# )

In [ ]:
# from transformers import pipeline
# import torch

# # 1. RETRIEVE from your ChromaDB (as we did before)
# user_query = "What should I do if I don't know the answer to a technical question?"
# results = collection.query(
#     query_texts=[user_query],
#     n_results=2 # Keep context short for local models
# )
# retrieved_context = " ".join(results['documents'][0])

# # 2. INITIALIZE LOCAL LLM (No API Key Required)
# # We use flan-t5-large because it's optimized for instructions
# print("⏳ Loading Flan-T5 model (approx 1.5GB)...")
# model_id = "google/flan-t5-large"
# generator = pipeline(
#     "text2text-generation",
#     model=model_id,
#     device=0 if torch.cuda.is_available() else -1  # Uses GPU if available
# )

# # 3. CONSTRUCT THE PROMPT (Matches your "Prompt Template" slide)
# # For T5, we use a specific instruction format
# input_text = f"answer the question using the context: {retrieved_context} Question: {user_query}"

# # 4. GENERATE
# output = generator(input_text, max_length=150, do_sample=False)

# print("\n🤖 LOCAL LLM RESPONSE (Flan-T5):")
# print(output[0]['generated_text'])

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Instantiate Retriever
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma(
    persist_directory="./interview_vector_db",
    embedding_function=embedding_model,
    collection_name = "interview_prep_collection"
)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2} # Fetches top-2 most relevant chunks
)

In [ ]:
test_docs = retriever.invoke("technical interview")
print(f"Success! Found {len(test_docs)} relevant chunks.")
if len(test_docs) > 0:
    print(f"Snippet: {test_docs[0].page_content[:100]}...")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Create Prompt Template
template = """
Use the following pieces of context to answer the question at the end.
If don't know the answer, say I don't know.
Context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template) #

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
import torch

# Initialize Free Local LLM (HuggingFace Flan-T5)
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    device = 0 if torch.cuda.is_available() else -1,
    pipeline_kwargs={
        "max_new_tokens": 200,
        "truncation": True,
        "max_length": 512
        }
)

# Assemble the Chain
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()} #
#     | prompt
#     | llm
#     | StrOutputParser() #
# )



# A helper function to extract ONLY the text content from retrieved documents
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Re-assemble the Chain
chain = (
    {
        "context": retriever | RunnableLambda(combine_docs), # Extracts only text
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
q = "What is the best way to handle a technical interview?"
a = chain.invoke(q)
print(a)

In [ ]:
test_questions = [
    "What is the best way to handle a technical interview?",
    "What do people on Reddit recommend doing if you get stuck on a coding question during a live interview?",
    "What are some common 'red flags' to look for in a company's data science culture according to interviewees?",
    "Based on the provided documents, how is the 'Bias-Variance Tradeoff' explained?",
    "What are the primary differences between supervised and unsupervised learning mentioned in the text?",
    "What are the three most common evaluation metrics for classification models according to the source?",
    "How do I explain my data science projects?",
    "What are the main mistakes candidates make in interviews?"
]

print("Running Batch Test...\n")

for q in test_questions:
    print(f"Question: {q}")
    answer = chain.invoke(q)
    print(f"Answer: {answer}")
    print("-" * 40)

In [ ]:
import json

def deep_clean_notebook(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        nb = json.load(f)

    # Remove the problematic widgets metadata entirely
    if 'metadata' in nb and 'widgets' in nb['metadata']:
        print("Found and removing metadata.widgets...")
        del nb['metadata']['widgets']

    # Also remove 'state' if it's hiding in the top level (rare but happens)
    if 'state' in nb.get('metadata', {}):
        del nb['metadata']['state']

    new_filename = filename.replace('.ipynb', '_fixed.ipynb')
    with open(new_filename, 'w', encoding='utf-8') as f:
        json.dump(nb, f, indent=2)

    print(f"Success! Please try opening: {new_filename}")

# Change this to your actual filename
deep_clean_notebook('InterviewRAG.ipynb')

In [ ]:
!git add --all

In [ ]:
!git commit -a -m "Fixed notebook"

In [ ]:
!git push origin main --force